#Tool-calling Agent

This is an auto-generated notebook created by an AI playground export. In this notebook, you will:
- Author a tool-calling [MLflow's `ResponsesAgent`](https://mlflow.org/docs/latest/api_reference/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ResponsesAgent) that uses the OpenAI client
- Manually test the agent's output
- Evaluate the agent with Mosaic AI Agent Evaluation
- Log and deploy the agent

This notebook should be run on serverless or a cluster with DBR<17.

 **_NOTE:_**  This notebook uses the OpenAI SDK, but AI Agent Framework is compatible with any agent authoring framework, including LlamaIndex or LangGraph. To learn more, see the [Authoring Agents](https://docs.databricks.com/generative-ai/agent-framework/author-agent) Databricks documentation.

## Prerequisites

- Address all `TODO`s in this notebook.

In [0]:
%pip install -U -qqqq backoff databricks-openai uv databricks-agents mlflow-skinny[databricks]
dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.12.1 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


## Define the agent in code
Below we define our agent code in a single cell, enabling us to easily write it to a local Python file for subsequent logging and deployment using the `%%writefile` magic command.

For more examples of tools to add to your agent, see [docs](https://docs.databricks.com/generative-ai/agent-framework/agent-tool.html).

In [0]:
%%writefile agent.py
import json
from typing import Any, Callable, Generator, Optional
from uuid import uuid4
import warnings

import backoff
import mlflow
import openai
from databricks.sdk import WorkspaceClient
from databricks_openai import UCFunctionToolkit, VectorSearchRetrieverTool
from mlflow.entities import SpanType, Document
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest,
    ResponsesAgentResponse,
    ResponsesAgentStreamEvent,
    output_to_responses_items_stream,
    to_chat_completions_input,
)
from openai import OpenAI
from pydantic import BaseModel
from unitycatalog.ai.core.base import get_uc_function_client

############################################
# Define your LLM endpoint and system prompt
############################################
LLM_ENDPOINT_NAME = "databricks-gpt-5-mini"

SYSTEM_PROMPT = """you are a helpful assistant, when prompted with a query search for related entries in the vector search. You report back with tables and datasets related to the user question."""


###############################################################################
## Define tools for your agent, enabling it to retrieve data or take actions
## beyond text generation
## To create and see usage examples of more tools, see
## https://docs.databricks.com/generative-ai/agent-framework/agent-tool.html
###############################################################################
class ToolInfo(BaseModel):
    """
    Class representing a tool for the agent.
    - "name" (str): The name of the tool.
    - "spec" (dict): JSON description of the tool (matches OpenAI Responses format)
    - "exec_fn" (Callable): Function that implements the tool logic
    - "is_retriever" (bool): Whether this tool is a retriever (for proper span typing)
    """

    name: str
    spec: dict
    exec_fn: Callable
    is_retriever: bool = False


def create_tool_info(tool_spec, exec_fn_param: Optional[Callable] = None, is_retriever: bool = False):
    tool_spec["function"].pop("strict", None)
    tool_name = tool_spec["function"]["name"]
    udf_name = tool_name.replace("__", ".")

    # Define a wrapper that accepts kwargs for the UC tool call,
    # then passes them to the UC tool execution client
    def exec_fn(**kwargs):
        function_result = uc_function_client.execute_function(udf_name, kwargs)
        if function_result.error is not None:
            return function_result.error
        else:
            return function_result.value
    return ToolInfo(name=tool_name, spec=tool_spec, exec_fn=exec_fn_param or exec_fn, is_retriever=is_retriever)


TOOL_INFOS = []

# You can use UDFs in Unity Catalog as agent tools
# TODO: Add additional tools
UC_TOOL_NAMES = []

uc_toolkit = UCFunctionToolkit(function_names=UC_TOOL_NAMES)
uc_function_client = get_uc_function_client()
for tool_spec in uc_toolkit.tools:
    TOOL_INFOS.append(create_tool_info(tool_spec))


# Use Databricks vector search indexes as tools
# See [docs](https://docs.databricks.com/generative-ai/agent-framework/unstructured-retrieval-tools.html) for details

# Use Databricks vector search indexes as tools
# See the [Databricks Documentation](https://docs.databricks.com/generative-ai/agent-framework/unstructured-retrieval-tools.html) for details
VECTOR_SEARCH_TOOLS = []
VECTOR_SEARCH_TOOLS.append(
        VectorSearchRetrieverTool(
            index_name="jack_demos_classic.data_catalogue_demo.metadata_docs_index",
            # TODO: specify index description for better agent tool selection
            # tool_description=""
        )
    )
for vs_tool in VECTOR_SEARCH_TOOLS:
    TOOL_INFOS.append(create_tool_info(vs_tool.tool, vs_tool.execute, is_retriever=True))



class ToolCallingAgent(ResponsesAgent):
    """
    Class representing a tool-calling Agent
    """

    def __init__(self, llm_endpoint: str, tools: list[ToolInfo]):
        """Initializes the ToolCallingAgent with tools."""
        self.llm_endpoint = llm_endpoint
        self.workspace_client = WorkspaceClient()
        self.model_serving_client: OpenAI = (
            self.workspace_client.serving_endpoints.get_open_ai_client()
        )
        self._tools_dict = {tool.name: tool for tool in tools}

    def get_tool_specs(self) -> list[dict]:
        """Returns tool specifications in the format OpenAI expects."""
        return [tool_info.spec for tool_info in self._tools_dict.values()]

    def execute_tool(self, tool_name: str, args: dict) -> Any:
        """Executes the specified tool with the given arguments."""
        tool_info = self._tools_dict[tool_name]
        
        # Use RETRIEVER span for retrieval tools, TOOL span for others
        span_type = SpanType.RETRIEVER if tool_info.is_retriever else SpanType.TOOL
        
        with mlflow.start_span(name=tool_name, span_type=span_type) as span:
            result = tool_info.exec_fn(**args)
            
            # For retriever tools, set the retrieved documents as span outputs
            if tool_info.is_retriever and result:
                try:
                    # Parse the result - VectorSearchRetrieverTool returns JSON string
                    if isinstance(result, str):
                        parsed_result = json.loads(result)
                    else:
                        parsed_result = result
                    
                    # Extract documents and convert to Document objects
                    documents = []
                    if isinstance(parsed_result, list):
                        for doc in parsed_result:
                            if isinstance(doc, dict):
                                # Extract page_content and metadata
                                page_content = doc.get("page_content", doc.get("content", doc.get("text", str(doc))))
                                metadata = doc.get("metadata", {})
                                doc_id = doc.get("id")
                                
                                # Create Document object
                                doc_obj = Document(
                                    page_content=page_content,
                                    metadata=metadata
                                )
                                if doc_id:
                                    doc_obj.id = doc_id
                                    
                                documents.append(doc_obj)
                    
                    # Set documents as span outputs for RAG scorers
                    if documents:
                        span.set_outputs(documents)
                        
                except Exception as e:
                    # If parsing fails, log but don't break the flow
                    print(f"Warning: Could not parse retriever results: {e}")
            
            return result

    def call_llm(self, messages: list[dict[str, Any]]) -> Generator[dict[str, Any], None, None]:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", message="PydanticSerializationUnexpectedValue")
            for chunk in self.model_serving_client.chat.completions.create(
                model=self.llm_endpoint,
                messages=to_chat_completions_input(messages),
                tools=self.get_tool_specs(),
                stream=True,
            ):
                chunk_dict = chunk.to_dict()
                if len(chunk_dict.get("choices", [])) > 0:
                    yield chunk_dict

    def handle_tool_call(
        self,
        tool_call: dict[str, Any],
        messages: list[dict[str, Any]],
    ) -> ResponsesAgentStreamEvent:
        """
        Execute tool calls, add them to the running message history, and return a ResponsesStreamEvent w/ tool output
        """
        try:
            args = json.loads(tool_call.get("arguments"))
        except Exception as e:
            args = {}
        result = str(self.execute_tool(tool_name=tool_call["name"], args=args))

        tool_call_output = self.create_function_call_output_item(tool_call["call_id"], result)
        messages.append(tool_call_output)
        return ResponsesAgentStreamEvent(type="response.output_item.done", item=tool_call_output)

    def call_and_run_tools(
        self,
        messages: list[dict[str, Any]],
        max_iter: int = 10,
    ) -> Generator[ResponsesAgentStreamEvent, None, None]:
        for _ in range(max_iter):
            last_msg = messages[-1]
            if last_msg.get("role", None) == "assistant":
                return
            elif last_msg.get("type", None) == "function_call":
                yield self.handle_tool_call(last_msg, messages)
            else:
                yield from output_to_responses_items_stream(
                    chunks=self.call_llm(messages), aggregator=messages
                )

        yield ResponsesAgentStreamEvent(
            type="response.output_item.done",
            item=self.create_text_output_item("Max iterations reached. Stopping.", str(uuid4())),
        )

    def predict(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        session_id = None
        if request.custom_inputs and "session_id" in request.custom_inputs:
            session_id = request.custom_inputs.get("session_id")
        elif request.context and request.context.conversation_id:
            session_id = request.context.conversation_id

        if session_id:
            mlflow.update_current_trace(
                metadata={
                    "mlflow.trace.session": session_id,
                }
            )

        outputs = [
            event.item
            for event in self.predict_stream(request)
            if event.type == "response.output_item.done"
        ]
        return ResponsesAgentResponse(output=outputs, custom_outputs=request.custom_inputs)

    def predict_stream(self, request: ResponsesAgentRequest) -> Generator[ResponsesAgentStreamEvent, None, None]:
        session_id = None
        if request.custom_inputs and "session_id" in request.custom_inputs:
            session_id = request.custom_inputs.get("session_id")
        elif request.context and request.context.conversation_id:
            session_id = request.context.conversation_id

        if session_id:
            mlflow.update_current_trace(
                metadata={
                    "mlflow.trace.session": session_id,
                }
            )

        messages = to_chat_completions_input([i.model_dump() for i in request.input])
        if SYSTEM_PROMPT:
            messages.insert(0, {"role": "system", "content": SYSTEM_PROMPT})
        yield from self.call_and_run_tools(messages=messages)


# Log the model using MLflow
mlflow.openai.autolog()
AGENT = ToolCallingAgent(llm_endpoint=LLM_ENDPOINT_NAME, tools=TOOL_INFOS)
mlflow.models.set_model(AGENT)

Overwriting agent.py


## Test the agent

Interact with the agent to test its output. Since we manually traced methods within `ResponsesAgent`, you can view the trace for each step the agent takes, with any LLM calls made via the OpenAI SDK automatically traced by autologging.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from agent import AGENT

AGENT.predict(
    {"input": [{"role": "user", "content": "what is 4*3 in python"}], "custom_inputs": {"session_id": "test-session-123"}},
)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:717)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
for chunk in AGENT.predict_stream(
    {"input": [{"role": "user", "content": "What is 4*3 in Python?"}], "custom_inputs": {"session_id": "test-session-123"}}
):
    print(chunk.model_dump(exclude_none=True))

{'type': 'response.output_item.done', 'item': {'type': 'function_call', 'id': 'chatcmpl-D9OZUGcu9WpLG0FVAIbPAN0oHiFlj', 'call_id': 'call_oeVGufSLXyLhy2TWunj8LuIz', 'name': 'jack_demos_classic__data_catalogue_demo__metadata_docs_index', 'arguments': '{"query":"4*3 in Python multiply 4 3 example code"}'}}
[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
{'type': 'response.output_item.done', 'item': {'type': 'function_call_output', 'call_id': 'call_oeVGufSLXyLhy2TWunj8LuIz', 'output': "[{'page_content': '# SUPPLY_DB.WAREHOUSE.INVENTORY_SNAPSHOT\\n\\n## Business Description\\nTable SUPPLY_DB.WAREHOUSE.INVENTORY_SNAPSHOT contains 4 modeled columns and supports domain analytics for supply_db.\\n\\n## User Comment\\nDaily inventory levels by SKU and location\\n\\n## Columns\\nON_HAND_QTY: On hand inventory quantity LOCATION_ID: Wareho

Trace(trace_id=tr-5832dada0411b07a6ba5c8e7343b48ae)

### Log the `agent` as an MLflow model
Determine Databricks resources to specify for automatic auth passthrough at deployment time
- **TODO**: If your Unity Catalog Function queries a [vector search index](https://docs.databricks.com/generative-ai/agent-framework/unstructured-retrieval-tools.html) or leverages [external functions](https://docs.databricks.com/generative-ai/agent-framework/external-connection-tools.html), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See [docs](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough) for more details.

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from agent import LLM_ENDPOINT_NAME, VECTOR_SEARCH_TOOLS, uc_toolkit
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from pkg_resources import get_distribution

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in VECTOR_SEARCH_TOOLS:
    resources.extend(tool.resources)
for tool in uc_toolkit.tools:
    # TODO: If the UC function includes dependencies like external connection or vector search, please include them manually.
    # See the TODO in the markdown above for more information.
    udf_name = tool.get("function", {}).get("name", "").replace("__", ".")
    resources.append(DatabricksFunction(function_name=udf_name))

input_example = {
    "input": [
        {
            "role": "user",
            "content": "crm"
        }
    ],
    "custom_inputs": {
        "session_id": "test-session"
    }
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        name="agent",
        python_model="agent.py",
        input_example=input_example,
        pip_requirements=[
            "databricks-openai",
            "backoff",
            f"databricks-connect=={get_distribution('databricks-connect').version}",
        ],
        resources=resources,
    )

/local_disk0/.ephemeral_nfs/envs/pythonEnv-b60e646a-3470-471e-8410-0bfe666485e5/lib/python3.10/site-packages/databricks/connect/session.py:451: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
🔗 View Logged Model at: https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/ml/experiments/2681661749963311/models/m-6ac37e5f5fbd4d35887dcc9befa1d3cc?o=1453025918090624
2026/02/15 05:10:44 INFO mlflow.pyfunc: Predicting on input example to validate output
2026/02/15 05:10:44 WARNING mlflow.tracing.fluent: No active trace found. Please create a span using `mlflow.start_span` or `@mlflow.trace` before calling `mlflow.update_current_trace`.
2026/02/15 05:10:44 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full tr

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


2026/02/15 05:10:44 WARNING mlflow.tracing.fluent: Failed to start span Completions: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2026/02/15 05:10:49 WARNING mlflow.tracing.fluent: No active trace found. Please create a span using `mlflow.start_span` or `@mlflow.trace` before calling `mlflow.update_current_trace`.
2026/02/15 05:10:49 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2026/02/15 05:10:49 WARNING mlflow.tracing.fluent: No active trace found. Please create a span using `mlflow.start_span` or `@mlflow.trace` before calling `mlflow.update_current_trace`.
2026/02/15 05:10:49 WARNING mlflow.tracing.fluent: Failed to start span Completions: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


2026/02/15 05:10:50 WARNING mlflow.tracing.fluent: Failed to start span Completions: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.


## Evaluate the agent with [Agent Evaluation](https://docs.databricks.com/mlflow3/genai/eval-monitor)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

Evaluate your agent with one of our [predefined LLM scorers](https://docs.databricks.com/mlflow3/genai/eval-monitor/predefined-judge-scorers), or try adding [custom metrics](https://docs.databricks.com/mlflow3/genai/eval-monitor/custom-scorers).

In [0]:
import mlflow
from mlflow.genai.scorers import RelevanceToQuery, Safety, RetrievalRelevance, RetrievalGroundedness, Correctness

eval_dataset = [
    {
        "inputs": {
            "input": [
                {
                    "role": "user",
                    "content": "Show me all tables related to CRM and sales opportunities"
                }
            ]
        },
        "expectations": {
            "expected_response": "should include the SALES_DB.CRM.OPPORTUNITIES table and related CRM tables"
        }
    }
]

eval_results = mlflow.genai.evaluate(
    data=eval_dataset,
    predict_fn=lambda input: AGENT.predict({"input": input, "custom_inputs": {"session_id": "evaluation-session"}}),
    scorers=[
        RelevanceToQuery(),           # Response addresses the query
        Safety(),                      # No harmful content
        Correctness(),                 # Matches expected response
        RetrievalRelevance(),          # Retrieved docs are relevant
        RetrievalGroundedness()        # Response grounded in retrieved context
    ],
)

# Review the evaluation results in the MLfLow UI (see console output)

2026/02/15 05:12:50 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset. To disable this check, set the MLFLOW_GENAI_EVAL_SKIP_TRACE_VALIDATION environment variable to True.
2026/02/15 05:12:50 WARNING mlflow.tracing.fluent: No active trace found. Please create a span using `mlflow.start_span` or `@mlflow.trace` before calling `mlflow.update_current_trace`.
2026/02/15 05:12:50 WARNING mlflow.tracing.fluent: Failed to start span predict_stream: 'NonRecordingSpan' object has no attribute 'context'. For full traceback, set logging level to debug.
2026/02/15 05:12:50 WARNING mlflow.tracing.fluent: No active trace found. Please create a span using `mlflow.start_span` or `@mlflow.trace` before calling `mlflow.update_current_trace`.


[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


Evaluating:   0%|          | 0/1 [Elapsed: 00:00, Remaining: ?] 

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.


## Perform pre-deployment validation of the agent
Before registering and deploying the agent, we perform pre-deployment checks via the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See [documentation](https://docs.databricks.com/machine-learning/model-serving/model-serving-debug.html#validate-inputs) for details

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"input": [{"role": "user", "content": "Hello!"}], "custom_inputs": {"session_id": "validation-session"}},
    env_manager="uv",
)

2026/02/15 05:14:12 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


2026/02/15 05:14:16 INFO mlflow.utils.virtualenv: Creating a new environment in /tmp/virtualenv_envs/mlflow-d30267116563d0a39185d9dff5c80b4fd4aa1a25 with python version 3.10.12 using uv
Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: /tmp/virtualenv_envs/mlflow-d30267116563d0a39185d9dff5c80b4fd4aa1a25
Activate with: source /tmp/virtualenv_envs/mlflow-d30267116563d0a39185d9dff5c80b4fd4aa1a25/bin/activate
2026/02/15 05:14:17 INFO mlflow.utils.virtualenv: Installing dependencies
Using Python 3.10.12 environment at: /tmp/virtualenv_envs/mlflow-d30267116563d0a39185d9dff5c80b4fd4aa1a25
Resolved 3 packages in 34ms
 Downloaded pip
Prepared 3 packages in 104ms
Installed 3 packages in 18ms
 + pip==22.3.1
 + setuptools==82.0.0
 + wheel==0.38.4
Using Python 3.10.12 environment at: /tmp/virtualenv_envs/mlflow-d30267116563d0a39185d9dff5c80b4fd4aa1a25
Resolved 150 packages in 923ms
 Downloaded tiktoken
 Downloaded kiwisolver
 Downloaded aiohttp
 Downloaded p

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True.
{"object": "response", "output": [{"type": "function_call", "id": "chatcmpl-D9OkisEcSGzKQnBebynW4i7hvOyfq", "call_id": "call_qfnX0fHq5tcpqlwyoeXw9llb", "name": "jack_demos_classic__data_catalogue_demo__metadata_docs_index", "arguments": "{\"query\":\"hello\"}"}, {"type": "function_call_output", "call_id": "call_qfnX0fHq5tcpqlwyoeXw9llb", "output": "[{'page_content': '# SALES_DB.CRM.OPPORTUNITIES\\n\\n## Business Description\\nTable SALES_DB.CRM.OPPORTUNITIES contains 4 modeled columns and supports domain analytics for sales_db.\\n\\n## User Comment\\nSales opportunities and pipeline status\\n\\n## Columns\\nEXPECTED_ARR: Expected annual recurring revenue STAGE: Current sales stage ACCOUNT_ID: Foreign key to account OPPORTUNITY_ID: Sales opportunity ID', 'metadata': {'doc_id': 'SA

2026/02/15 05:14:34 INFO mlflow.tracing.export.async_export_queue: Flushing the async trace logging queue before program exit. This may take a while...


## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "jack_demos_classic"
schema = "data_catalogue_demo"
model_name = "catalog_rag_agent"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

Successfully registered model 'jack_demos_classic.data_catalogue_demo.catalog_rag_agent'.


Uploading artifacts:   0%|          | 0/13 [00:00<?, ?it/s]

🔗 Created version '1' of model 'jack_demos_classic.data_catalogue_demo.catalog_rag_agent': https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/explore/data/models/jack_demos_classic/data_catalogue_demo/catalog_rag_agent/version/1?o=1453025918090624


## Deploy the agent

In [0]:
from databricks import agents
# NOTE: pass scale_to_zero=True to agents.deploy() to enable scale-to-zero for cost savings.
# This is not recommended for production workloads, as capacity is not guaranteed when scaled to zero.
# Scaled to zero endpoints may take extra time to respond when queried, while they scale back up.
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "playground"})

/local_disk0/.ephemeral_nfs/envs/pythonEnv-b60e646a-3470-471e-8410-0bfe666485e5/lib/python3.10/site-packages/databricks/agents/deployments.py:641: UserWarning: This endpoint is being deployed without a feedback model, which has been deprecated.
For more information, see: https://docs.databricks.com/aws/en/generative-ai/agent-framework/feedback-model
  warnings.warn(



    Deployment of jack_demos_classic.data_catalogue_demo.catalog_rag_agent version 1 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/ml/endpoints/agents_jack_demos_classic-data_catalogue_demo-catalog_rag_agent/?o=1453025918090624
    Review App: https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/ml/review-v2/46b81ce779bf40b1a1efe718e9a08dde/chat?o=1453025918090624

You can refer back to the links above from the endpoint detail page at https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/ml/endpoints/agents_jack_demos_classic-data_catalogue_demo-catalog_rag_agent/?o=1453025918090624.

To set up monitoring for your deployed agent, see:
https://docs.databricks.com/aws/en/mlflow3/genai/eval-monitor/production-monitoring


Deployment(model_name='jack_demos_classic.data_catalogue_demo.catalog_rag_agent', model_version='1', endpoint_name='agents_jack_demos_classic-data_catalogue_demo-catalog_rag_agent', served_entity_name='jack_demos_classic-data_catalogue_demo-catalog_rag_agent_1', query_endpoint='https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/serving-endpoints/agents_jack_demos_classic-data_catalogue_demo-catalog_rag_agent/served-models/jack_demos_classic-data_catalogue_demo-catalog_rag_agent_1/invocations?o=1453025918090624', endpoint_url='https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/ml/endpoints/agents_jack_demos_classic-data_catalogue_demo-catalog_rag_agent/?o=1453025918090624', review_app_url='https://fe-vm-vdm-classic-eq33dc.cloud.databricks.com/ml/review-v2/46b81ce779bf40b1a1efe718e9a08dde/chat?o=1453025918090624')

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See [docs](https://docs.databricks.com/generative-ai/deploy-agent.html) for details